<a href="https://colab.research.google.com/github/5amessi/Binary-Classification/blob/master/Binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import math
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
from keras.models import Sequential
from keras import regularizers
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.optimizers import RMSprop
from keras.initializers import glorot_uniform, glorot_normal, RandomUniform
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from datetime import timedelta
from sklearn.externals import joblib 
import datetime

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
def Clean(data,valid = False):
  data2 = data.index
  data = data.values
  new_data = []
  for i in range(len(data)):
    if(str(data[i][0]) == str(1e400*0)):
      pass
    else:
      new_line1 = list(map(str,data2[i][0].split(';')))
      new_line2 = list(map(str,data2[i][1].split(';')))
      new_line3 = list(map(str,data2[i][2].split(';')))      
      new_line4 = list(map(str,data[i][0].split(';')))
      
      new_data.append(new_line1 + new_line2 + new_line3 + new_line4)
#       print(new_line1 + new_line2 + new_line3 + new_line4)
  data = pd.DataFrame(new_data,columns=['v0','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v18','v18','v19','v20','target'])
  data.to_csv("data.csv",index=False)
  data = pd.read_csv("/content/data.csv")

  return data


In [41]:
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/binary_classification/training.csv")
data = Clean(data)
data.head()


,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v18,v18.1,v19,v20,target
0,a,17,92,5,0.00004,"""u""","""g""","""c""","""v""",1,75,"""f""","""t""",1,"""t""","""g""",80.0,5,800000.0,"""t""",0,"""no."""
1,b,16,92,3,0.00035,"""y""","""p""","""k""","""v""",0,29,"""f""","""f""",0,"""f""","""s""",200.0,0,2000000.0,NaN,0,"""no."""
2,a,48,17,0,1335.00000,"""u""","""g""","""i""","""o""",0,335,"""f""","""f""",0,"""f""","""g""",0.0,120,0.0,NaN,0,"""no."""
3,b,32,33,0,35.00000,"""u""","""g""","""k""","""v""",0,5,"""f""","""f""",0,"""t""","""g""",232.0,0,2320000.0,"""f""",0,"""no."""
4,a,34,83,0,125.00000,"""y""","""p""","""i""","""h""",0,5,"""f""","""f""",0,"""t""","""g""",160.0,0,1600000.0,"""f""",0,"""no."""


In [43]:
for i in data:
  if (len(data[i].unique()) < 20):
    print(i,data[i].unique())

v0 ['a' 'b' nan]
v2 [92 17 33 83 75 58 42 67  5 25  8]
v3 [5 3 0 8 7 6 4 1 2 9]
v5 ['"u"' '"y"']
v6 ['"g"' '"p"']
v7 ['"c"' '"k"' '"i"' '"q"' '"W"' '"ff"' '"d"' '"cc"' '"r"' '"m"' '"e"'
 '"aa"' '"j"' '"x"' nan]
v8 ['"v"' '"o"' '"h"' '"ff"' '"bb"' '"n"' '"j"' nan]
v9 [ 1  0  2  5  4  6  3  8 14  7  9 12 17 28 13 15]
v11 ['"f"' '"t"']
v12 ['"t"' '"f"']
v13 [ 1  0  2  3  5 11 12  9 14  4  6  7 16 15  8 67 40 10]
v14 ['"t"' '"f"']
v15 ['"g"' '"s"' '"p"']
v19 ['"t"' nan '"f"']
v20 [0 1]
target ['"no."' '"yes."']


In [0]:
def Prepare(data):
  data['v0'] = data['v0'].fillna(50)
  data['v7'] = data['v7'].fillna(50)
  data['v8'] = data['v8'].fillna(50)
  data['v19'] = data['v19'].fillna(50)
  
  for i in range(len(data['v0'].unique())):
    data['v0'] = data['v0'].replace(data['v0'].unique()[i],i)
  
  for i in range(len(data['v5'].unique())):
    data['v5'] = data['v5'].replace(data['v5'].unique()[i],i)

  for i in range(len(data['v6'].unique())):
    data['v6'] = data['v6'].replace(data['v6'].unique()[i],i)

  for i in range(len(data['v7'].unique())):
    data['v7'] = data['v7'].replace(data['v7'].unique()[i],i)

  for i in range(len(data['v8'].unique())):
    data['v8'] = data['v8'].replace(data['v8'].unique()[i],i)
    
  for i in range(len(data['v11'].unique())):
    data['v11'] = data['v11'].replace(data['v11'].unique()[i],i)
    
  for i in range(len(data['v12'].unique())):
    data['v12'] = data['v12'].replace(data['v12'].unique()[i],i)
    
  for i in range(len(data['v14'].unique())):
    data['v14'] = data['v14'].replace(data['v14'].unique()[i],i)
    
  for i in range(len(data['v15'].unique())):
    data['v15'] = data['v15'].replace(data['v15'].unique()[i],i)
    
  for i in range(len(data['v19'].unique())):
    data['v19'] = data['v19'].replace(data['v19'].unique()[i],i)
    
  for i in range(len(data['target'].unique())):
    data['target'] = data['target'].replace(data['target'].unique()[i],i)
  data = data.fillna(data.mean())
  return data

In [45]:
data = Prepare(data)
data.head()

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v18,v18.1,v19,v20,target
0,0,17,92,5,0.00004,0,0,0,0,1,75,0,0,1,0,0,80.0,5,800000.0,0,0,0
1,1,16,92,3,0.00035,1,1,1,0,0,29,0,1,0,1,1,200.0,0,2000000.0,1,0,0
2,0,48,17,0,1335.00000,0,0,2,1,0,335,0,1,0,1,0,0.0,120,0.0,1,0,0
3,1,32,33,0,35.00000,0,0,1,0,0,5,0,1,0,0,0,232.0,0,2320000.0,2,0,0
4,0,34,83,0,125.00000,1,1,2,2,0,5,0,1,0,0,0,160.0,0,1600000.0,2,0,0


In [29]:
y = data['target']
x = data.drop(['target'],axis=1)
x.shape,y.shape

((2009, 21), (2009,))

In [30]:
scaler = StandardScaler()
x = scaler.fit_transform(x)
x.shape

(2009, 21)

In [0]:
activ_func = 'tanh'  
loss = 'binary_crossentropy'                  
optimizer="adam"              
dropout = 0.05                 

In [32]:
model = Sequential()
model.add(Dense(256,input_dim=np.shape(x)[1],activation=activ_func,kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(128,activation=activ_func))
model.add(Dense(units=1,activation = 'sigmoid'))
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

W0623 07:03:10.018068 140227782694784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0623 07:03:10.039593 140227782694784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0623 07:03:10.042729 140227782694784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0623 07:03:10.092494 140227782694784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0623 07:03:10.118707 140227782694784 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               5632      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(np.array(x), np.array(y), epochs=1, batch_size=128, verbose=1)

Epoch 1/1
2009/2009 [==============================] - 0s 35us/step - loss: 0.0575 - acc: 1.0000


In [34]:
valid = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/binary_classification/validation.csv")
valid = Clean(valid)
valid = Prepare(valid)
print(valid.head())
valid_y = valid['target']
valid_x = valid.drop(['target'],axis=1)
valid_x.shape,valid_y.shape

   v0  v1  v2  v3           v4  v5  ...  v16  v18      v18.1  v19  v20  target
0   0  32  33   0     75.00000   0  ...  420    0  4200000.0    0    1       0
1   0  23  58   0    179.00000   0  ...  136    1  1360000.0    0    0       0
2   0  36  42   7      0.00005   1  ...  240    3  2400000.0    0    1       0
3   0  18  42   0  10415.00000   1  ...  120  375  1200000.0    0    0       0
4   0  24   5   0  13335.00000   1  ...  120  475  1200000.0    1    1       0

[5 rows x 22 columns]


((125, 21), (125,))

In [39]:
from sklearn.metrics import precision_recall_fscore_support
y_pred = model.predict_classes(valid_x)
result = precision_recall_fscore_support(valid_y, y_pred,average='binary')
print('precision=',result[0],'recall=',result[1], 'f1score=',result[2])

precision= 0.4426229508196721 recall= 0.9818181818181818 f1score= 0.6101694915254238
